In [ ]:
import matplotlib.pyplot as plt
import pickle
import numpy as np

method = "LN_COBYLA (NLopt)"
problem = "maxcut"
depth_pool = [1, 4]
qubit_pool = list(range(10, 15, 2))
n = 10
seed_pool = list(range(100))

for p in depth_pool:
    result, initial_ar, max_ar = [], [], []
    for i, n in enumerate(qubit_pool):
        # data = pickle.load(
        #     open(
        #         f"data/{problem}/configs/max_ar/{method}-p{p}-q{n}-s0-999.pckl",
        #         "rb",
        #     ),
        # )
        # max_ar.append(data["result"][:len(seed_pool)])

        data = pickle.load(
            open(
                f"data/{problem}/configs/rhobeg/{method}-p{p}-q{n}-s{seed_pool[0]}-{seed_pool[-1]}.pckl",
                "rb",
            ),
        )
        config = data["config"]
        maxfev_pool = config["maxeval"]
        rhobeg_pool = config["initial_step"]
        print(p, n)
        result.append(data["result"])
        initial_ar.append(data["initial_ar"])
    result = np.array(result)
    initial_ar = np.array(initial_ar)[:,:,np.newaxis,np.newaxis]
    # max_ar = np.array(max_ar)[:,:,np.newaxis,np.newaxis]
    max_ar = result[:,:,-1,0][:,:,np.newaxis,np.newaxis]
    result = (result - initial_ar) / (max_ar - initial_ar)
    print(config)

    plt.figure(figsize=(12, 4))
    bar_width = 0.3
    colors = ["tab:blue", "tab:orange", "pink", "lightblue", "lightgreen", "violet"]
    for j, (maxfev, c) in enumerate(zip(maxfev_pool, colors)):
        ys = np.array(rhobeg_pool)[np.argmax(result[:,:,j], axis=2)]
        print(np.median(ys[0]), np.mean(ys[0]))
        plt.violinplot(
            ys.T,
            # sym="+",
            positions=np.array(qubit_pool)+bar_width*j,
            widths=bar_width,
            showmeans=True,
            # showmedians=True,
        )
        plt.plot([], c=c, label=f"{maxfev=}")
    plt.xticks(np.array(qubit_pool) + bar_width * (len(maxfev_pool)-1) / 2, labels=qubit_pool)
    plt.ylabel("rhobeg")
    plt.title(f"{p=}")
    plt.legend()

    plt.figure(figsize=(12, 10))
    for j, maxfev in enumerate(maxfev_pool[:-1]):
        ys = result[:,:,j]
        plt.errorbar(rhobeg_pool, np.mean(ys, axis=(0,1)), yerr=np.std(ys, axis=(0,1))/np.sqrt(len(seed_pool)), label=f"{maxfev=}")

    # plt.ylim((0.9, 1))
    plt.ylabel("AR")
    plt.xlabel("rhobeg")
    plt.title(f"{p=}")
    plt.legend()
